### AI model init 

In [22]:
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI
import os

from dotenv import load_dotenv
load_dotenv()

# if "GOOGLE_API_KEY" not in os.environ:
#     os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

# Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


# Local Ollama Model 
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage

# Replace this with your local Ollama model name
MODEL_NAME = "llama2"

# Initialize the Ollama chat model
# llm = ChatOllama(
#     model=MODEL_NAME,
#     base_url="http://localhost:11434",  # Default Ollama server URL
#     temperature=0,                   # Optional: Adjust sampling temperature
#     max_retries=3                      # Optional: Number of retries for failed requests
# )


# Example usage
# response = llm([HumanMessage(content="Explain the theory of relativity.")])
# print(response.content)


### init custom LLMGraphTransformer and neo4j instance 

In [23]:
from src.graph_builder.llm import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)



import os
from dotenv import load_dotenv
from src.graph_builder.lc_neo4j import Neo4jGraph
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "iambatman"
load_dotenv()
query_vector = llm_transformer._embed_text("")

graph = Neo4jGraph(refresh_schema=True,
                embedding_dim = len(query_vector)
)


Init LLMGraphTransformer


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.33it/s]



INIT Neo4jGraph -5


### Read document and split and convert to GraphDocuments

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# 1) Read your full text (e.g. from PDF)
full_text = open("op.txt", encoding="utf-8").read()

# 2) Wrap in a single Document
original_doc = Document(page_content=full_text, metadata={"doc_id":"mydoc"})

# 3) Configure the splitter
splitter = RecursiveCharacterTextSplitter(
    # target chunk size in characters (not tokens)
    chunk_size=500,
    # how many characters of overlap between chunks
    chunk_overlap=100,
    # what to split on first, second, etc.
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
)

# 4) Produce a list of Documents, each with its own chunk
chunked_docs = splitter.split_documents([original_doc])

# Inspect
for i, doc in enumerate(chunked_docs[:3]):
    print(f"--- chunk {i} ({len(doc.page_content)} chars) ---")
    print(doc.page_content[:200].replace("\n"," "), "…\n")
print(len(chunked_docs))


graph_documents = llm_transformer.convert_to_graph_documents(chunked_docs)


--- chunk 0 (472 chars) ---
Neuro Prune: An Adaptive Approach for Efficient    Deep Neural Network Optimization on Edge    Devices    Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5    1Pro …

--- chunk 1 (444 chars) ---
sanjayjayakumar91@gmail.com , suryanarayanaant@gmail.com    Abstract—The optimization of deep neural networks for de-    ployment on edge devices is a significant research area due to    the demand fo …

--- chunk 2 (446 chars) ---
computational power, energy consumption, and latency. Pruning    is a method to reduce the model size, accelerate inference, and    save power. The objective of the paper is to propose the Neuro    Pr …

6
In -- convert_to_graph_documents


Batches: 100%|██████████| 1/1 [00:05<00:00,  5.51s/it]

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.34it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.54it/s]



Merging Nodes
3) Merge relationships
4) Compute embeddings for each node now that snippets are final


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.82it/s]


5) embeding relationships


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.66it/s]

Building final GraphDocument


### print pretty graph

In [25]:
def pretty_print_graph_documents(graph_documents):
    for idx, graph_doc in enumerate(graph_documents, start=1):
        print(f"\n### GraphDocument {idx}")
        print("**Nodes:**")
        for node_idx, node in enumerate(graph_doc.nodes, start=1):
            print(f"\n  {node_idx}. **Node ID:** {node.id}")
            print(f"     **Type:** {node.type}")
            print(f"     **Properties:**")
            for key, value in node.properties.items():
                if key == "source_texts":
                    print(f"       - **{key.capitalize()}:**")
                    for text in value:
                        print(f"         ```\n         {text}\n         ```")
                else:
                    print(f"       - **{key.capitalize()}:** {value}")

# Assuming `graph_documents` is the variable holding the output
pretty_print_graph_documents(graph_documents)
print(len(graph_documents))


### GraphDocument 1
**Nodes:**

  1. **Node ID:** b85404dfb4f434847a08e5ac0c17e749
     **Type:** Passage
     **Properties:**
       - **Text:** Neuro Prune: An Adaptive Approach for Efficient
   Deep Neural Network Optimization on Edge
   Devices
   Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5
   1Professor, Department of Computer Science and Engineering, PSG College of Technology
   grk.cse@psgtech.ac.in
   2,3,4,5BE-CSE (AI&ML), PSG College of Technology
   adhishthesak@gmail.com, mohanakumarp2828@gmail.com
   sanjayjayakumar91@gmail.com , suryanarayanaant@gmail.com
       - **Source_doc_id:** mydoc
       - **Source_page:** None
       - **Embedding:** [-0.012503384612500668, 0.06143878772854805, -0.00673447223380208, 0.025237757712602615, 0.014757942408323288, 0.033177461475133896, -0.017098233103752136, -0.005254596937447786, -0.03517308086156845, 0.026807203888893127, 0.027776500210165977, 0.001669572084210813, -0.035905372351408005, 0

### Add GraphDocuments to DB

In [26]:
graph.setup_graph_with_documents(graph_documents)

Vector index 'passageEmbeddings' created for node label 'Passage' on property 'embedding' with dimension 768 using COSINE similarity.
Vector index 'algorithmEmbeddings' created for node label 'Algorithm' on property 'embedding' with dimension 768 using COSINE similarity.
Vector index 'modelEmbeddings' created for node label 'Model' on property 'embedding' with dimension 768 using COSINE similarity.
Vector index 'organizationEmbeddings' created for node label 'Organization' on property 'embedding' with dimension 768 using COSINE similarity.
Vector index 'deviceEmbeddings' created for node label 'Device' on property 'embedding' with dimension 768 using COSINE similarity.
Vector index 'methodEmbeddings' created for node label 'Method' on property 'embedding' with dimension 768 using COSINE similarity.
Vector index 'conceptEmbeddings' created for node label 'Concept' on property 'embedding' with dimension 768 using COSINE similarity.
Vector index 'departmentEmbeddings' created for node lab

In [27]:
from src.graph_builder.llm import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)



import os
from dotenv import load_dotenv
from src.graph_builder.lc_neo4j import Neo4jGraph
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "iambatman"
load_dotenv()
query_vector = llm_transformer._embed_text("")

graph = Neo4jGraph(refresh_schema=True,
                embedding_dim = len(query_vector)
)


Init LLMGraphTransformer


Batches: 100%|██████████| 1/1 [00:00<00:00,  5.05it/s]



INIT Neo4jGraph -5


In [28]:
for doc in graph_documents:
    print(doc.relationships[0].properties['embedding'])   # Ensure embeddings are unique for each document
    print(doc.relationships[1].properties['embedding'])   # Ensure embeddings are unique for each document
    print(doc.relationships[2].properties['embedding'])   # Ensure embeddings are unique for each document


[-0.029524635523557663, 0.02821071818470955, -0.03181152418255806, 0.03932451456785202, -0.003279311815276742, -0.047103989869356155, 0.0496368370950222, 0.020443812012672424, -0.033717602491378784, 0.013925912790000439, -0.01716557703912258, 0.015231272205710411, -0.004539158660918474, 0.021805698052048683, -0.017529137432575226, 0.01560961827635765, 0.020639600232243538, 0.05385364219546318, 0.01659226045012474, -0.014793611131608486, -0.012477925978600979, -0.007249241229146719, 0.014675354585051537, 0.031223254278302193, -0.024090008810162544, 0.004843361210078001, 0.030760934576392174, 0.04833907261490822, 0.02213960327208042, -0.0086794663220644, 0.003023585770279169, 0.003872302360832691, 0.0006754331407137215, 0.010425611399114132, 2.4064411263680086e-06, 0.02582680433988571, 0.039550263434648514, 0.019349820911884308, -0.020786702632904053, 0.01306375116109848, 0.056766726076602936, -0.00025836200802586973, -0.0009719912777654827, -0.0014797807671129704, -0.011479047127068043,

In [29]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "iambatman"))

def get_all_node_embeddings(tx):
    query = "MATCH (n) WHERE n.embedding is NOT NULL RETURN id(n) AS id, n.embedding AS embedding, n.source_texts as source_texts"
    return list(tx.run(query))

def get_all_relationship_embeddings(tx):
    query = "MATCH ()-[r]->() WHERE r.embedding is NOT NULL RETURN id(r) AS id, r.embedding AS embedding,  r.source_texts as source_texts"
    return list(tx.run(query))

with driver.session() as session:
    node_results = session.read_transaction(get_all_node_embeddings)
    rel_results = session.read_transaction(get_all_relationship_embeddings)


C:\Users\SURYA\AppData\Local\Temp\ipykernel_14216\2219753826.py:15: DeprecationWarning: read_transaction has been renamed to execute_read
  node_results = session.read_transaction(get_all_node_embeddings)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 48, offset: 47} for query: 'MATCH (n) WHERE n.embedding is NOT NULL RETURN id(n) AS id, n.embedding AS embedding, n.source_texts as source_texts'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 48, offset: 47} for q

In [30]:
len(node_results)

116

In [31]:
import numpy as np

# Suppose you have your query embedding as a numpy array
query_embedding = np.array([...])  # Your query embedding
query_text = "Surya Narayanaa N T"

query_embedding = llm_transformer._embed_text(query_text)

# Search across all node types
results = graph.search_similar_nodes(query_embedding)

# Or search relationships
rel_results = graph.search_similar_relationships(query_embedding,print_results=False)

# def cosine_similarity(a, b):
#     a = np.array(a)
#     b = np.array(b)
#     return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# # For nodes
# node_similarities = []
# for record in node_results:
#     node_id = record["id"]
#     embedding = record["embedding"]
#     txt = record["source_texts"]
#     sim = cosine_similarity(query_embedding, embedding)
#     node_similarities.append((node_id, sim, txt))

# # For relationships
# rel_similarities = []
# for record in rel_results:
#     rel_id = record["id"]
#     embedding = record["embedding"]
#     sim = cosine_similarity(query_embedding, embedding)
#     rel_similarities.append((rel_id, sim))

# # Sort and get top-k
# top_k = 10
# top_nodes = sorted(node_similarities, key=lambda x: x[1], reverse=True)[:top_k]
# top_rels = sorted(rel_similarities, key=lambda x: x[1], reverse=True)[:top_k]


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.19it/s]




Top 5 results for Database:
  No results found

Top 5 results for Documentation:
  No results found

Top 5 results for Index:
  No results found

Top 5 results for Library:
  No results found

Top 5 results for Movie:
  No results found

Top 5 results for Number:
  No results found

Top 5 results for Organization:
  1. [0.551] Psg College Of Technology

Top 5 results for Test:
  No results found

Top 5 results for Tutorial:
  No results found

Top 5 results for Version:
  No results found

Top 5 results for Algorithm:
  1. [0.508] Neuro Prune Algorithm

Top 5 results for Application:
  1. [0.514] Smart Cameras
  2. [0.514] Autonomous Navigation
  3. [0.514] Augmented Reality
  4. [0.478] Healthcare Applications

Top 5 results for Concept:
  1. [0.525] Ai Deployment
  2. [0.522] Efficiency
  3. [0.522] Subnetworks
  4. [0.522] Sparsity
  5. [0.522] Tunable Parameter

Top 5 results for Department:
  1. [0.551] Department Of Computer Science And Engineering

Top 5 results for Device:
  1

In [32]:
results 

{'Database': [],
 'Documentation': [],
 'Index': [],
 'Library': [],
 'Movie': [],
 'Number': [],
 'Organization': [{'item': {'embedding': [-0.029524635523557663,
     0.02821071818470955,
     -0.03181152418255806,
     0.03932451456785202,
     -0.003279311815276742,
     -0.047103989869356155,
     0.0496368370950222,
     0.020443812012672424,
     -0.033717602491378784,
     0.013925912790000439,
     -0.01716557703912258,
     0.015231272205710411,
     -0.004539158660918474,
     0.021805698052048683,
     -0.017529137432575226,
     0.01560961827635765,
     0.020639600232243538,
     0.05385364219546318,
     0.01659226045012474,
     -0.014793611131608486,
     -0.012477925978600979,
     -0.007249241229146719,
     0.014675354585051537,
     0.031223254278302193,
     -0.024090008810162544,
     0.004843361210078001,
     0.030760934576392174,
     0.04833907261490822,
     0.02213960327208042,
     -0.0086794663220644,
     0.003023585770279169,
     0.003872302360832691,
 

In [33]:
# Function to print top k results
def print_top_k_results(results, k=5):
    """Print the top k results for each type from search results."""
    for type_name, type_results in results.items():
        print(f"\nTop {k} results for {type_name}:")
        if not type_results:
            print("  No results found")
            continue
            
        # Sort results by score in descending order
        sorted_results = sorted(type_results, key=lambda x: x['score'], reverse=True)
        for i, result in enumerate(sorted_results[:k]):
            item = result['item']
            
            # Get display property - prefer text/content/name/id
            display_props = ['text', 'content', 'name', 'id']
            display_value = next(
                (str(item.get(p)) for p in display_props if item.get(p) is not None),
                str(item.get('id', 'No ID'))
            )
            
            # Truncate long values
            if len(display_value) > 100:
                display_value = display_value[:97] + "..."
            
            # Print result with score
            print(f"  {i+1}. [{result['score']:.3f}] {display_value}")

def get_top_k_across_all_types(results, k=5):
    """Get the top k results across all node types combined.
    
    Args:
        results: Dict mapping node types to their search results
        k: Number of top results to return
        
    Returns:
        List of tuples (score, node_type, display_value)
    """
    all_results = []
    
    for type_name, type_results in results.items():
        if not type_results:
            continue
            
        for result in type_results:
            item = result['item']
            score = result['score']
            
            # Get display property - prefer text/content/name/id
            display_props = ['text', 'content', 'name', 'id', 'source_texts']
            display_value = next(
                (str(item.get(p)) for p in display_props if item.get(p) is not None),
                str(item.get('id', 'No ID'))
            )
            
            all_results.append((score, type_name, display_value))
    
    # Sort by score in descending order and get top k
    top_k_results = sorted(all_results, key=lambda x: x[0], reverse=True)[:k]
    
    # Print formatted results
    print("\nTop", k, "results across all types:")
    for score, type_name, display_value in top_k_results:
        # Truncate long values
        if len(display_value) > 100:
            display_value = display_value[:97] + "..."
        print(f"  [{score:.3f}] ({type_name}) {display_value}")
        
    return top_k_results

# Get the top 5 results across all types
# top_5_overall = get_top_k_across_all_types(results, k=5)

# # Create a formatted string that can be passed to an LLM
# llm_input = "Top 5 most relevant results:\n" + "\n".join([
#     f"{i+1}. [{score:.3f}] ({type_name}) {value}" 
#     for i, (score, type_name, value) in enumerate(top_5_overall)
# ])

In [34]:
results

{'Database': [],
 'Documentation': [],
 'Index': [],
 'Library': [],
 'Movie': [],
 'Number': [],
 'Organization': [{'item': {'embedding': [-0.029524635523557663,
     0.02821071818470955,
     -0.03181152418255806,
     0.03932451456785202,
     -0.003279311815276742,
     -0.047103989869356155,
     0.0496368370950222,
     0.020443812012672424,
     -0.033717602491378784,
     0.013925912790000439,
     -0.01716557703912258,
     0.015231272205710411,
     -0.004539158660918474,
     0.021805698052048683,
     -0.017529137432575226,
     0.01560961827635765,
     0.020639600232243538,
     0.05385364219546318,
     0.01659226045012474,
     -0.014793611131608486,
     -0.012477925978600979,
     -0.007249241229146719,
     0.014675354585051537,
     0.031223254278302193,
     -0.024090008810162544,
     0.004843361210078001,
     0.030760934576392174,
     0.04833907261490822,
     0.02213960327208042,
     -0.0086794663220644,
     0.003023585770279169,
     0.003872302360832691,
 

In [35]:
def get_top_k_with_sources(results, k=5):
    """Get the top k results across all node types with their source information.
    
    Args:
        results: Dict mapping node types to their search results
        k: Number of top results to return
        
    Returns:
        List of tuples (score, node_type, display_value, source_texts)
    """
    all_results = []
    
    for type_name, type_results in results.items():
        if not type_results:
            continue
            
        for result in type_results:
            item = result['item']
            score = result['score']
            
            # Get basic properties
            display_props = ['text', 'content', 'name', 'id']
            display_value = next(
                (str(item.get(p)) for p in display_props if item.get(p) is not None),
                str(item.get('id', 'No ID'))
            )
            
            # Get source information if available
            source_texts = item.get('source_texts', [])
            if not isinstance(source_texts, list):
                source_texts = [str(source_texts)]
            
            all_results.append((score, type_name, display_value, source_texts))
    
    # Sort by score in descending order and get top k
    top_k_results = sorted(all_results, key=lambda x: x[0], reverse=True)[:k]
    
    # Print formatted results
    print("\nTop", k, "results across all types:")
    for score, type_name, display_value, source_texts in top_k_results:
        print(f"\n[{score:.3f}] ({type_name}) {display_value}")
        if source_texts:
            print("Source:")
            for text in source_texts:
                print(textwrap.fill(text, width=100, initial_indent="  ", subsequent_indent="  "))
        
    return top_k_results

# Import textwrap for pretty printing
import textwrap

# Get the top 5 results across all types with sources
top_5_with_sources = get_top_k_with_sources(results, k=5)

# Create a formatted string for LLM use
llm_input = "Top 5 most relevant results with sources:\n\n" + "\n\n".join([
    f"{i+1}. [{score:.3f}] ({type_name}) {value}\nSource:\n" + "\n".join(f"  {text}" for text in sources)
    for i, (score, type_name, value, sources) in enumerate(top_5_with_sources)
])


Top 5 results across all types:

[0.609] (Passage) model has an accuracy increase of 0.22%.
   I. INTRODUCTION
   The rise of intelligent systems, ranging from autonomous
   drones to augmented reality (AR) devices and smart surveil-
   lance cameras, has intensified the need for efficient deep neural
   network deployment on edge devices with constrained com-
   putational resources. While advanced deep learning models
   like YOLO, SSD, ResNet, and MobileNet excel in accuracy,
   their deployment on such devices is hindered by significant

[0.609] (Passage) their deployment on such devices is hindered by significant
   computational demands, large memory footprints, and high
   energy consumption. These challenges pose critical barriers to
   real-time applications and sustainable deployment.
   This paper builds on existing pruning techniques, includ-
   ing unstructured pruning, structured pruning, and approaches
   inspired by quantization, highlighting their strengths and lim-



In [53]:
def format_rag_prompt(results, user_query):
    """Format search results and create a structured prompt for Gemini.
    
    Args:
        results: Dict mapping node types to their search results
        user_query: The original user query
        
    Returns:
        Tuple[str, str]: System prompt and formatted context
    """
    # First get top results with sources
    all_results = []
    
    for type_name, type_results in results.items():
        if not type_results:
            continue
            
        for result in type_results:
            item = result['item']
            score = result['score']
            
            # Get basic properties
            display_props = ['text', 'content', 'name', 'id']
            display_value = next(
                (str(item.get(p)) for p in display_props if item.get(p) is not None),
                str(item.get('id', 'No ID'))
            )
            
            # Get source information
            source_texts = item.get('source_texts', [])
            if not isinstance(source_texts, list):
                source_texts = [str(source_texts)]
            
            all_results.append((score, type_name, display_value, source_texts))
    
    # Sort by score and get top results
    top_results = sorted(all_results, key=lambda x: x[0], reverse=True)[:5]
    
    # Create system prompt
    system_prompt = f"""You are a specialized RAG (Retrieval Augmented Generation) agent designed to:
1. Answer questions using ONLY the provided context
2. Maintain academic accuracy and precision
3. Cite your sources when providing information
4. Admit when you don't have enough information to answer
5. Focus on the most relevant information from the highest-scoring matches
6. Consider both the content and the type of each retrieved item

USER QUERY : {user_query}
REMEMBER THIS WELL: Analyze the user query first, if it has some general data that is not provided in the context of retrieved item, try to repond to the user GENERALLY 

Base your responses ONLY on the retrieved context provided below. """

    # Format context
    context = "Retrieved Context (sorted by relevance):\n\n"
    for i, (score, type_name, value, sources) in enumerate(top_results):
        context += f"[Result {i+1}] Score: {score:.3f} | Type: {type_name}\n"
        context += f"Item: {value}\n"
        if sources:
            context += "Source:\n"
            for text in sources:
                context += textwrap.fill(text, width=100, initial_indent="  ", subsequent_indent="  ") + "\n"
        context += "\n"

    return system_prompt, context

# Import required modules
import textwrap
user_query = "   Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5"
# Get the search results formatted for RAG
system_prompt, context = format_rag_prompt(results, user_query)

# Format the complete prompt for Gemini
gemini_input = f"""{system_prompt}

{context}

User Query: {user_query}

Please provide a detailed answer based on the retrieved information above."""

print("Generated prompt for Gemini:")
print("-" * 80)
print(gemini_input)
print("-" * 80)

# The gemini_input variable is now ready to be sent to the Gemini API

Generated prompt for Gemini:
--------------------------------------------------------------------------------
You are a specialized RAG (Retrieval Augmented Generation) agent designed to:
1. Answer questions using ONLY the provided context
2. Maintain academic accuracy and precision
3. Cite your sources when providing information
4. Admit when you don't have enough information to answer
5. Focus on the most relevant information from the highest-scoring matches
6. Consider both the content and the type of each retrieved item

USER QUERY :    Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5
REMEMBER THIS WELL: Analyze the user query first, if it has some general data that is not provided in the context of retrieved item, try to repond to the user GENERALLY 

Base your responses ONLY on the retrieved context provided below. 

Retrieved Context (sorted by relevance):

[Result 1] Score: 0.609 | Type: Passage
Item: model has an accuracy increase of 0.22%

In [54]:
def query_gemini(prompt):
    """Send a prompt to Gemini API and get the response.
    
    Args:
        prompt: The formatted prompt to send to Gemini
        
    Returns:
        str: Gemini's response
    """
    # We already have ChatGoogleGenerativeAI initialized as 'llm'
    # No need to reinitialize the client
    try:
        # Use the existing llm instance to generate response
        response = llm.invoke(prompt)
        
        # Return the response content
        return response.content
        
    except Exception as e:
        return f"Error querying Gemini: {str(e)}"

# Test the function with the existing gemini_input
try:
    response = query_gemini(gemini_input)
    
    print("\nGemini's Response:")
    print("-" * 80)
    print(response)
    print("-" * 80)
except Exception as e:
    print(f"Error: {str(e)}")



Gemini's Response:
--------------------------------------------------------------------------------
I am sorry, but the provided context does not contain any information about Dr. Karpagam G, Adhish Krishna S, Mohana Kumar P, Sanjay J, or Surya Narayanaa N T. Therefore, I cannot provide a detailed answer about them.

Generally, the suffixes R1, S2, P3, J4, and T5 might refer to affiliations or positions of these individuals within a research group or organization. Without additional context, it's impossible to determine their specific roles or contributions.
--------------------------------------------------------------------------------
